Checkers

In [1]:
import math
from array import *
from copy import deepcopy
from functools import partial

class checkers:
    
    # Player pieces are denoted 1 (pawn) and 2 (king), opponents are negative
    def __init__(self, player = 1, position = [1] * 12 + [0] * 8 + [-1] * 12):
        
        if player == 1:
            self.position = position
        else:
            self.position = [-position[31-i] for i in range(32)]
            
        self.pawns = []
        self.kings = []
        self.epawns = []
        self.ekings = []
        
        self.type = "all"
        self.actions = []
        
        self.win = 0
        self.winMove = []
        self.notlMove = []
        self.optimalActions = []
        
    # Find location and count of all pieces
    def updateLocation(self):
        
        pawns = []
        kings = []
        epawns = []
        ekings = []
        
        for index, piece in enumerate(self.position):
            if piece == 1:
                pawns.append(index)
            elif piece == 2:
                kings.append(index)
            elif piece == -1:
                epawns.append(index)
            elif piece == -2:
                ekings.append(index)
        
        self.pawns = pawns
        self.kings = kings
        self.epawns = epawns
        self.ekings = ekings
        
    # Find all possible actions
    def actions(pawns, kings, epawns, ekings, key):
        
        # Whenever a branch is made, the input will be in the form [piece size (1/2), location of piece moved, piece jumped over, ending location]
        jumps = []
        
        # jumps have length 4, regular moves have length 3
        moves = []
        
        if key == None:
            
            # Check if a pawn jump can be made
            for location in pawns:
                
                if (location // 4) % 2 == 0:
                    
                    if location % 4 in [0,1,2]:
                        
                        if (((location + 5) in epawns) or ((location + 5) in ekings)) and (not (((location + 9) in epawns) or ((location + 9) in ekings) or ((location + 9) in pawns) or ((location + 9) in kings))) and (location + 9 < 32):
                                                    
                            jumps.append([1, location, 5, 9])
                        
                    if location % 4 in [1,2,3]:
                        
                        if (((location + 4) in epawns) or ((location + 4) in ekings)) and (not (((location + 7) in epawns) or ((location + 7) in ekings) or ((location + 7) in pawns) or ((location + 7) in kings))) and (location + 7 < 32):
                                                    
                            jumps.append([1, location, 4, 7])

                else:
                    
                    if location % 4 in [0,1,2]:
                        
                        if (((location + 4) in epawns) or ((location + 4) in ekings)) and (not (((location + 9) in epawns) or ((location + 9) in ekings)) or ((location + 9) in pawns) or ((location + 9) in kings)) and (location + 9 < 32):
                                                    
                            jumps.append([1, location, 4, 9])
                        
                    if location % 4 in [1,2,3]:
                        
                        if (((location + 3) in epawns) or ((location + 3) in ekings)) and (not (((location + 7) in epawns) or ((location + 7) in ekings) or ((location + 7) in pawns) or ((location + 7) in kings))) and (location + 7 < 32):
                                                    
                            jumps.append([1, location, 3, 7])

            # Check if a king jump can be made
            for location in kings:
                    
                if (location // 4) % 2 == 0:
                    
                    if location % 4 in [0,1,2]:
                        
                        if (((location + 5) in epawns) or ((location + 5) in ekings)) and (not (((location + 9) in epawns) or ((location + 9) in ekings) or ((location + 9) in pawns) or ((location + 9) in kings))) and (location + 9 < 32):
                                                    
                            jumps.append([2, location, 5, 9])
                        
                        if (((location - 3) in epawns) or ((location - 3) in ekings)) and (not (((location - 7) in epawns) or ((location - 7) in ekings) or ((location - 7) in pawns) or ((location - 7) in kings))) and (location - 7 >= 0):
                                                    
                            jumps.append([2, location, -3, -7])
                        
                    if location % 4 in [1,2,3]:
                        
                        if (((location + 4) in epawns) or ((location + 4) in ekings)) and (not (((location + 7) in epawns) or ((location + 7) in ekings) or ((location + 7) in pawns) or ((location + 7) in kings))) and (location + 7 < 32):
                                                    
                            jumps.append([2, location, 4, 7])

                        if (((location - 4) in epawns) or ((location - 4) in ekings)) and (not (((location - 9) in epawns) or ((location - 9) in ekings) or ((location - 9) in pawns) or ((location - 9) in kings))) and (location - 9 >= 0):
                                                    
                            jumps.append([2, location, -4, -9])
                            
                else:
                    
                    if location % 4 in [0,1,2]:
                        
                        if (((location + 4) in epawns) or ((location + 4) in ekings)) and (not (((location + 9) in epawns) or ((location + 9) in ekings) or ((location + 9) in pawns) or ((location + 9) in kings))) and (location + 9 < 32):
                                                    
                            jumps.append([2, location, 4, 9])

                        if (((location - 4) in epawns) or ((location - 4) in ekings)) and (not (((location - 7) in epawns) or ((location - 7) in ekings) or ((location - 7) in pawns) or ((location - 7) in kings))) and (location - 7 >= 0):
                                                    
                            jumps.append([2, location, -4, -7])
                        
                    if location % 4 in [1,2,3]:
                        
                        if (((location + 3) in epawns) or ((location + 3) in ekings)) and (not (((location + 7) in epawns) or ((location + 7) in ekings) or ((location + 7) in pawns) or ((location + 7) in kings))) and (location + 7 < 32):
                                                    
                            jumps.append([2, location, 3, 7])
                            
                        if (((location - 5) in epawns) or ((location - 5) in ekings)) and (not (((location - 9) in epawns) or ((location - 9) in ekings) or ((location - 9) in pawns) or ((location - 9) in kings))) and (location - 9 >= 0):
                                                    
                            jumps.append([2, location, -5, -9])
                            
            # If a jump can be made:
            if len(jumps) > 0:
                
                for jump in jumps:
                    
                    # If the jump promotes the pawn
                    if jump[0] == 1 and (jump[1] + jump[3]) // 4 == 7:
                        
                        moves.append([jump])
                    
                    else:
                        
                        newpawns = deepcopy(pawns)
                        newkings = deepcopy(kings)
                        newepawns = deepcopy(epawns)
                        newekings = deepcopy(ekings)
                        
                        if jump[0] == 1:                            
                            newpawns.remove(jump[1])
                        else:
                            newkings.remove(jump[1])

                        if (jump[1] + jump[2]) in epawns:
                            newepawns.remove(jump[1] + jump[2])
                        
                        else:
                            newekings.remove(jump[1] + jump[2])
                                
                        if jump[0] == 1:
                            newpawns.append(jump[1] + jump[3])
                        else:
                            newkings.append(jump[1] + jump[3])
                        
                        key = jump[1] + jump[3]
                        
                        newactions = checkers.actions(newpawns, newkings, newepawns, newekings, key)
                        moves += [[jump] + action for action in newactions]
                        
            # No jumps, so only normal moves
            else:
                
                # Check if a pawn step can be made
                for location in pawns:
                    
                    if (location // 4) % 2 == 0:
                        
                        if location % 4 in [0,1,2]:
                            
                            if not (((location + 5) in epawns) or ((location + 5) in ekings) or ((location + 5) in pawns) or ((location + 5) in kings)) and (location + 5 < 32):
                                                        
                                moves.append([[1, location, 5], [""]])
                            
                        if location % 4 in [0,1,2,3]:
                            
                            if not (((location + 4) in epawns) or ((location + 4) in ekings) or ((location + 4) in pawns) or ((location + 4) in kings)) and (location + 4 < 32):
                                                        
                                moves.append([[1, location, 4], [""]])

                    else:
                        
                        if location % 4 in [0,1,2,3]:
                            
                            if not (((location + 4) in epawns) or ((location + 4) in ekings) or ((location + 4) in pawns) or ((location + 4) in kings)) and (location + 4 < 32):
                                                        
                                moves.append([[1, location, 4], [""]])
                            
                        if location % 4 in [1,2,3]:
                            
                            if not (((location + 3) in epawns) or ((location + 3) in ekings) or ((location + 3) in pawns) or ((location + 3) in kings)) and (location + 3 < 32):
                                                        
                                moves.append([[1, location, 3], [""]])

                # Check if a king step can be made
                for location in kings:
                        
                    if (location // 4) % 2 == 0:
                        
                        if location % 4 in [0,1,2]:
                            
                            if not (((location + 5) in epawns) or ((location + 5) in ekings) or ((location + 5) in pawns) or ((location + 5) in kings)) and (location + 5 < 32):
                                                        
                                moves.append([[2, location, 5], [""]])

                            if not (((location - 3) in epawns) or ((location - 3) in ekings) or ((location - 3) in pawns) or ((location - 3) in kings)) and (location - 3 >= 0):
                                                        
                                moves.append([[2, location, -3], [""]])
                                
                        if location % 4 in [0,1,2,3]:
                            
                            if not (((location + 4) in epawns) or ((location + 4) in ekings) or ((location + 4) in pawns) or ((location + 4) in kings)) and (location + 4 < 32):
                                                        
                                moves.append([[2, location, 4], [""]])
                            
                            if not (((location - 4) in epawns) or ((location - 4) in ekings) or ((location - 4) in pawns) or ((location - 4) in kings)) and (location - 4 >= 0):
                                                        
                                moves.append([[2, location, -4], [""]])
                                
                    else:
                        
                        if location % 4 in [0,1,2,3]:
                            
                            if not (((location + 4) in epawns) or ((location + 4) in ekings) or ((location + 4) in pawns) or ((location + 4) in kings)) and (location + 4 < 32):
                                                        
                                moves.append([[2, location, 4], [""]])

                            if not (((location - 4) in epawns) or ((location - 4) in ekings) or ((location - 4) in pawns) or ((location - 4) in kings)) and (location - 4 >= 0):
                                                        
                                moves.append([[2, location, -4], [""]])

                            
                        if location % 4 in [1,2,3]:
                            
                            if not (((location + 3) in epawns) or ((location + 3) in ekings) or ((location + 3) in pawns) or ((location + 3) in kings)) and (location + 3 < 32):
                                                        
                                moves.append([[2, location, 3], [""]])

                            if not (((location - 5) in epawns) or ((location - 5) in ekings) or ((location - 5) in pawns) or ((location - 5) in kings)) and (location - 5 >= 0):
                                                        
                                moves.append([[2, location, -5], [""]])

        # There was a previous jump
        else:
            
            if key in pawns:
                                
                if (key // 4) % 2 == 0:
                    
                    if key % 4 in [0,1,2]:
                        
                        if (((key + 5) in epawns) or ((key + 5) in ekings)) and (not (((key + 9) in epawns) or ((key + 9) in ekings) or ((key + 9) in pawns) or ((key + 9) in kings))) and (key + 9 < 32):
                                                    
                            jumps.append([1, key, 5, 9])
                        
                    if key % 4 in [1,2,3]:
                        
                        if (((key + 4) in epawns) or ((key + 4) in ekings)) and (not (((key + 7) in epawns) or ((key + 7) in ekings) or ((key + 7) in pawns) or ((key + 7) in kings))) and (key + 7 < 32):
                                                    
                            jumps.append([1, key, 4, 7])

                else:
                    
                    if key % 4 in [0,1,2]:
                        
                        if (((key + 4) in epawns) or ((key + 4) in ekings)) and (not (((key + 9) in epawns) or ((key + 9) in ekings) or ((key + 9) in pawns) or ((key + 9) in kings))) and (key + 9 < 32):
                                                    
                            jumps.append([1, key, 4, 9])
                        
                    if key % 4 in [1,2,3]:
                        
                        if (((key + 3) in epawns) or ((key + 3) in ekings)) and (not (((key + 7) in epawns) or ((key + 7) in ekings) or ((key + 7) in pawns) or ((key + 7) in kings))) and (key + 7 < 32):
                                                    
                            jumps.append([1, key, 3, 7])

            # Check if a king jump can be made
            if key in kings:
                    
                if (key // 4) % 2 == 0:
                    
                    if key % 4 in [0,1,2]:
                        
                        if (((key + 5) in epawns) or ((key + 5) in ekings)) and (not (((key + 9) in epawns) or ((key + 9) in ekings) or ((key + 9) in pawns) or ((key + 9) in kings))) and (key + 9 < 32):
                                                    
                            jumps.append([2, key, 5, 9])
                        
                        if (((key - 3) in epawns) or ((key - 3) in ekings)) and (not (((key - 7) in epawns) or ((key - 7) in ekings) or ((key - 7) in pawns) or ((key - 7) in kings))) and (key - 7 >= 0):
                                                    
                            jumps.append([2, key, -3, -7])
                        
                    if key % 4 in [1,2,3]:
                        
                        if (((key + 4) in epawns) or ((key + 4) in ekings)) and (not (((key + 7) in epawns) or ((key + 7) in ekings) or ((key + 7) in pawns) or ((key + 7) in kings))) and (key + 7 < 32):
                                                    
                            jumps.append([2, key, 4, 7])
                        
                        if (((key - 4) in epawns) or ((key - 4) in ekings)) and (not (((key - 9) in epawns) or ((key - 9) in ekings) or ((key - 9) in pawns) or ((key - 9) in kings))) and (key - 9 >= 0):
                                                    
                            jumps.append([2, key, -4, -9])
                            
                else:
                    
                    if key % 4 in [0,1,2]:
                        
                        if (((key + 4) in epawns) or ((key + 4) in ekings)) and (not (((key + 9) in epawns) or ((key + 9) in ekings) or ((key + 9) in pawns) or ((key + 9) in kings))) and (key + 9 < 32):
                                                    
                            jumps.append([2, key, 4, 9])
                        
                        if (((key - 4) in epawns) or ((key - 4) in ekings)) and (not (((key - 7) in epawns) or ((key - 7) in ekings) or ((key - 7) in pawns) or ((key - 7) in kings))) and (key - 7 >= 0):
                                                    
                            jumps.append([2, key, -4, -7])
                        
                    if key % 4 in [1,2,3]:
                        
                        if (((key + 3) in epawns) or ((key + 3) in ekings)) and (not (((key + 7) in epawns) or ((key + 7) in ekings) or ((key + 7) in pawns) or ((key + 7) in kings))) and (key + 7 < 32):
                                                    
                            jumps.append([2, key, 3, 7])

                        if (((key - 5) in epawns) or ((key - 5) in ekings)) and (not (((key - 9) in epawns) or ((key - 9) in ekings) or ((key - 9) in pawns) or ((key - 9) in kings))) and (key - 9 >= 0):
                                                    
                            jumps.append([2, key, -5, -9])
                                         
            if len(jumps) > 0:
                
                for jump in jumps:
                    
                    # If the jump promotes the pawn
                    if jump[0] == 1 and (jump[1] + jump[3]) // 4 == 7:
                        
                        moves.append([jump])
                    
                    else:
                        
                        newpawns = deepcopy(pawns)
                        newkings = deepcopy(kings)
                        newepawns = deepcopy(epawns)
                        newekings = deepcopy(ekings)
                        
                        if jump[0] == 1:                            
                            newpawns.remove(jump[1])
                        else:
                            newkings.remove(jump[1])

                        if (jump[1] + jump[2]) in epawns:
                            newepawns.remove(jump[1] + jump[2])
                        
                        else:
                            newekings.remove(jump[1] + jump[2])
                                
                        if jump[0] == 1:
                            newpawns.append(jump[1] + jump[3])
                        else:
                            newkings.append(jump[1] + jump[3])
                        
                        key = jump[1] + jump[3]
                        
                        newactions = checkers.actions(newpawns, newkings, newepawns, newekings, key)
                        moves += [[jump] + action for action in newactions]
        
        if len(moves) == 0:
            moves = [[[""]]]
        
        return(moves)

    # Update actions with actions()
    def updateActions(self):
        self.updateLocation()
        self.actions = checkers.actions(self.pawns, self.kings, self.epawns, self.ekings, None)
    
    def opponentAction(self):
        
        fpawns = [31-i for i in self.epawns]
        fkings = [31-i for i in self.ekings]
        fepawns = [31-i for i in self.pawns]
        fekings = [31-i for i in self.kings]
        
        fposition = []
        for index in range(32):
            if index in fpawns:
                fposition.append(1)
            elif index in fkings:
                fposition.append(2)
            elif index in fepawns:
                fposition.append(-1)
            elif index in fekings:
                fposition.append(-2)
            else:
                fposition.append(0)
        
        fsubposition = checkers(position = fposition)
        return(fsubposition.optimalActions)
    
    # REDUNDANT TO flip()??? MIGHT DELETE
    def opponentAction(pawns, kings, epawns, ekings):
        
        fpawns = [31-i for i in epawns]
        fkings = [31-i for i in ekings]
        fepawns = [31-i for i in pawns]
        fekings = [31-i for i in kings]
        
        fposition = []
        for index in range(32):
            if index in fpawns:
                fposition.append(1)
            elif index in fkings:
                fposition.append(2)
            elif index in fepawns:
                fposition.append(-1)
            elif index in fekings:
                fposition.append(-2)
            else:
                fposition.append(0)
        
        fsubposition = checkers(position = fposition)
        #print(fsubposition.optimalActions)
        return(fsubposition.optimalActions)
        
    # Update board position after an action
    def updateBoard(self, action):
        
        #print(action)
        
        self.updateActions()
        
        pawns = deepcopy(self.pawns)
        kings = deepcopy(self.kings)
        epawns = deepcopy(self.epawns)
        ekings = deepcopy(self.ekings)
        
        #print()
        #print(pawns)
        #print(kings)
        #print(epawns)
        #print(ekings)
        #print()
        
        """
        position = []
        for index in range(32):
            if index in pawns:
                position.append(1)
            elif index in kings:
                position.append(2)
            elif index in epawns:
                position.append(-1)
            elif index in ekings:
                position.append(-2)
            else:
                position.append(0)
        
        subposition = checkers(position = position)
        subposition.updateLocation()
        print(subposition.moves)
        """

        # print(action)
        
        for step in action:
            
            # print(step)
                        
            if step == [""]:
                
                break
            
            elif len(step) == 3:
                
                if step[0] == 1:

                    pawns.remove(step[1])
                    
                    if (step[1] + step[2]) // 4 == 7:
                        kings.append(step[1] + step[2])
                        
                    else:
                        pawns.append(step[1] + step[2])
                    
                else:
                    
                    kings.remove(step[1])
                    kings.append(step[1] + step[2])
                
            else:
                
                if step[0] == 1:
                    
                    pawns.remove(step[1])
                    
                    if (step[1] + step[2]) in epawns:
                        epawns.remove(step[1] + step[2])
                    else:
                        ekings.remove(step[1] + step[2])
                    
                    if (step[1] + step[3]) // 4 == 7:
                        kings.append(step[1] + step[3])
                    else:
                        pawns.append(step[1] + step[3])

                else:
                                        
                    kings.remove(step[1])

                    if (step[1] + step[2]) in epawns:
                        epawns.remove(step[1] + step[2])
                    else:
                        ekings.remove(step[1] + step[2])
                        
                    kings.append(step[1] + step[3])
                
        board = []
        for space in range(32):
            if space in pawns:
                board.append(1)
            elif space in kings:
                board.append(2)
            elif space in epawns:
                board.append(-1)
            elif space in ekings:
                board.append(-2)
            else:
                board.append(0)
        
        self.position = board
        self.pawns = pawns
        self.kings = kings
        self.epawns = epawns
        self.ekings = ekings

    # Print out the board
    def displayBoard(self):
        
        board = [str(i) for i in deepcopy(self.position)]
        
        print("-\t" + "\t-\t".join(board[0:4]).replace("0", "-") + "\n")
        print("\t-\t".join(board[4:8]) .replace("0", "-")+ "\t- \n")
        print("-\t" + "\t-\t".join(board[8:12]).replace("0", "-") + "\n")
        print("\t-\t".join(board[12:16]).replace("0", "-") + "\t- \n")
        print("-\t" + "\t-\t".join(board[16:20]).replace("0","-") + "\n")
        print("\t-\t".join(board[20:24]).replace("0", "-") + "\t- \n")
        print("-\t" + "\t-\t".join(board[24:28]).replace("0", "-") + "\n")
        print("\t-\t".join(board[28:32]).replace("0", "-") + "\t- \n")
    
    # See if the game is won
    def winCheck(pawns, kings, epawns, ekings):

        if len(epawns) + len(ekings) == 0:
            return(True)
        
        # Flipping the board and seeing if there are any enemy moves (if no moves possible, game is won)
        fpawns = [31-i for i in epawns]
        fkings = [31-i for i in ekings]
        fepawns = [31-i for i in pawns]
        fekings = [31-i for i in kings]
        
        if [''] in checkers.actions(fpawns, fkings, fepawns, fekings, None):
            return(True)
        
        return(False)
    
    def winUpdate(self):
        
        if checkers.winCheck(self.pawns, self.kings, self.epawns, self.ekings):

            self.win = 1
            return(True)

        return(False)
    
    def winningMove(self):
        
        for action in self.actions:
            
            #print("winning move")
            #print(action)
            
            subposition = checkers(position = self.position)
            subposition.updateBoard(action = action)
            if subposition.winUpdate():
                self.winMove = [action]
                self.win = 1
                return(action)
        
        self.winMove = [[[""]]]
        return([[""]])
        
    def blockWinningMove(self):
        
        validActions = []
        
        for action in self.actions:
            
            subposition = checkers(position = self.position)
            subposition.updateBoard(action = action)
            
            fpawns = [31-i for i in subposition.epawns]
            fkings = [31-i for i in subposition.ekings]
            fepawns = [31-i for i in subposition.pawns]
            fekings = [31-i for i in subposition.kings]
            
            subposition.pawns = fpawns
            subposition.kings = fkings
            subposition.epawns = fepawns
            subposition.ekings = fekings
            
            fposition = []
            for index in range(32):
                if index in fpawns:
                    fposition.append(1)
                elif index in fkings:
                    fposition.append(2)
                elif index in fepawns:
                    fposition.append(-1)
                elif index in fekings:
                    fposition.append(-2)
                else:
                    fposition.append(0)
            
            fsubposition = checkers(position = fposition)
            
            if [""] in fsubposition.winningMove():
                validActions.append(action)
        
        if len(validActions) == 0:
            validActions = self.actions

        self.notlMove = validActions
        self.optimalActions = validActions
        return(validActions)
    
    def flip(pawns, kings, epawns, ekings):
        
        fpawns = [31-i for i in epawns]
        fkings = [31-i for i in ekings]
        fepawns = [31-i for i in pawns]
        fekings = [31-i for i in kings]
        
        fposition = []
        for index in range(32):
            if index in fpawns:
                fposition.append(1)
            elif index in fkings:
                fposition.append(2)
            elif index in fepawns:
                fposition.append(-1)
            elif index in fekings:
                fposition.append(-2)
            else:
                fposition.append(0)
        
        fsubposition = checkers(position = fposition)
        return(fsubposition)

    def updateAll(self):
        self.updateActions()
        # print(self.actions)
        
        self.winUpdate()
        self.winningMove()
        self.blockWinningMove()
        
        if not self.winMove == [[[""]]]:
            self.optimalActions = self.winMove

UCB

In [2]:
import random
import math
from array import *
from copy import deepcopy
import numpy as np

graphs = []

class ucb:
    
    def __init__(self, position = [1] * 12 + [0] * 8 + [-1] * 12, explore_param = math.sqrt(2), depth = 5):
        
        self.checkers = checkers(position = position)
        self.checkers.updateAll()
        
        self.children = {}
        
        self.position = []
        self.explore_param = explore_param
        
        self.total_visits = 0
        self.num_visits = {}
        self.depth = depth
        
        self.UCBVals = {}
        self.qBar = {}
        self.qHat = 0
        
        self.vHat = []
        self.optimalActions = []
                       
    def opponentAction(pawns, kings, epawns, ekings):
        
        flipState = checkers.flip(pawns, kings, epawns, ekings)
        flipState.updateAll()
        # print(flipState.optimalActions)
        return(random.choice(flipState.optimalActions))

    def tuplefy(move):
        
        newmove = []
        
        for step in move:
            
            newmove.append(tuple(step))
            
            if step == [""]:
                break
            
        return(tuple(newmove))
    
    def untuplefy(move):

        flag = True
        newmove = []
        
        for step in move:
            
            if list(step) == [""]:
                break
            
            newmove.append(list(step))
            
        newmove.append([""])
        
        return(newmove)
    
    def update(self, move, depth):
        
        """
        if isinstance(move[0], int):
            move = [move]    
           """
            
        if self.checkers.win == 1:
            return(1)
                
        position = deepcopy(self.checkers)
        position.updateBoard(move)

        opponentMove = ucb.opponentAction(position.pawns, position.kings, position.epawns, position.ekings)
        opponentBoard = checkers.flip(position.pawns, position.kings, position.epawns, position.ekings)
        opponentBoard.updateBoard(opponentMove)
        subposition = checkers.flip(opponentBoard.pawns, opponentBoard.kings, opponentBoard.epawns, opponentBoard.ekings)
        subposition.updateAll()

        if depth > 1:
            
            #print("blah")
                
            if ucb.tuplefy(move) in self.children:
                
                childState = self.children[ucb.tuplefy(move)]
                childState.position = subposition.position
                childState.checkers = checkers(position = childState.position)
                childState.checkers.updateAll()
            
            else:
            
                childState = ucb(position = subposition.position, explore_param = self.explore_param, depth = self.depth-1)
                self.children[ucb.tuplefy(move)] = childState
            
            self.total_visits += 1
            
            childMove = childState.starvingAction()
            """
            if isinstance(childMove[0], int):
                childMove = [childMove]
            """
             
            qval = childState.update(childMove, depth - 1)
            
            if ucb.tuplefy(move) in self.UCBVals:
                self.qBar[ucb.tuplefy(move)] = (self.qBar[ucb.tuplefy(move)] * self.num_visits[ucb.tuplefy(move)] + qval)/(self.num_visits[ucb.tuplefy(move)]+1)
                self.num_visits[ucb.tuplefy(move)] += 1
                # self.UCBVals[ucb.tuplefy(move)] = self.qBar[ucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits))
                for move in self.UCBVals:
                    self.UCBVals[move] = self.qBar[ucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits)/self.num_visits[move])
                    # print(move)
                
            else:
                self.qBar[ucb.tuplefy(move)] = qval
                self.num_visits[ucb.tuplefy(move)] = 1
                # self.UCBVals[ucb.tuplefy(move)] = self.qBar[ucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits))
                self.UCBVals[ucb.tuplefy(move)] = self.qBar[ucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits)/self.num_visits[ucb.tuplefy(move)])
                
            for i in self.qBar:
                if self.qBar[i] > self.qHat:
                    self.qHat = self.qBar[i]
                    
            return(self.qHat)
        
        else:
            
            return(ucb.rollout(subposition))
            
    def rollout(position):
        
        board = checkers(position)
        
        for iter in range(40):
                
            board.updateAll()
            if board.win == 1:
                return(1)
    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
            board.updateAll()
            if board.win == 1:
                return(-1)    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
        
        board.updateLocation()
        # print(board.pawns, board.kings, board.epawns, board.ekings)
        return((len(board.pawns) + 1.5 * len(board.kings))/(len(board.pawns) + len(board.epawns) + 1.5 * len(board.kings) + 1.5 * len(board.ekings)))
    
    def starvingAction(self):
        
        maxUCB = 0
        maxAction = []
        
        for action in self.checkers.optimalActions:
                        
            if not (ucb.tuplefy(action) in self.children):
                
                return(action)

            if self.UCBVals[ucb.tuplefy(action)] > maxUCB:
                
                maxUCB = self.UCBVals[ucb.tuplefy(action)] 
                maxAction = action
        
        return(maxAction)
    
    def simulate(self):
        
        # print()
        # print(self.checkers.optimalActions)
        # print()
               
        if self.checkers.win == 1:
            
            return(self.checkers.optimalActions[0])
        
        for val in range(100):
            
            self.update(self.starvingAction(), self.depth)

        optscore = 0
        optmove = 0
        for i in self.qBar:
            if self.qBar[i] > optscore:
                optscore = self.qBar[i]
                optmove = i

        return(optmove)
            
    def traverse(self):

        temp = []
        temp.append(self.total_visits)
        
        if len(self.children) > 0:

            for child in self.children:
                
                temp.append(ucb.traverse(self.children[child]))
        print(temp)
        return(temp)

BUCB

In [3]:
import random
import math
from array import *
from copy import deepcopy
import numpy as np

graphs = []

class bucb:
    
    def __init__(self, position = [1] * 12 + [0] * 8 + [-1] * 12, explore_param = math.sqrt(2), depth = 5):
        
        self.checkers = checkers(position = position)
        self.checkers.updateAll()
        
        self.children = {}
        
        self.position = []
        self.explore_param = explore_param
        
        self.total_visits = 0
        self.num_visits = {}
        self.depth = depth
        
        self.UCBVals = {}
        self.qBar = {}
        self.qHat = 0
        
        self.vHat = []
        
        self.actions = []
        self.optimalActions = []
                        
    def opponentAction(pawns, kings, epawns, ekings):
        
        flipState = checkers.flip(pawns, kings, epawns, ekings)
        flipState.updateAll()
        return(random.choice(flipState.optimalActions))
    
    # todo: Move tuplefy out of ucb
    def tuplefy(move):
        
        newmove = []
        
        for step in move:
            
            newmove.append(tuple(step))
            
            if step == [""]:
                break
            
        return(tuple(newmove))
    
    def untuplefy(move):

        flag = True
        newmove = []
        
        for step in move:
            
            if list(step) == [""]:
                break
            
            newmove.append(list(step))
            
        newmove.append([""])
        
        return(newmove)
        
    def update(self, move, depth):

        if self.checkers.win == 1:
            return(1)
                
        position = deepcopy(self.checkers)
        position.updateBoard(move)

        opponentMove = bucb.opponentAction(position.pawns, position.kings, position.epawns, position.ekings)

        opponentBoard = checkers.flip(position.pawns, position.kings, position.epawns, position.ekings)
        opponentBoard.updateBoard(opponentMove)
        subposition = checkers.flip(opponentBoard.pawns, opponentBoard.kings, opponentBoard.epawns, opponentBoard.ekings)
        subposition.updateAll()

        if depth > 1:
                    
            if bucb.tuplefy(move) in self.children:
                
                childState = self.children[bucb.tuplefy(move)]
                childState.position = subposition.position
                childState.checkers = checkers(position = childState.position)
                childState.checkers.updateAll()
            
            else:
            
                childState = bucb(position = subposition.position, explore_param = self.explore_param, depth = self.depth-1)
                self.children[bucb.tuplefy(move)] = childState
            
            self.total_visits += 1
            
            childMove = childState.starvingAction()
            
            qval = childState.update(childMove, depth - 1)
            
            if bucb.tuplefy(move)in self.UCBVals:
                self.qBar[bucb.tuplefy(move)] = (self.qBar[bucb.tuplefy(move)] * self.num_visits[bucb.tuplefy(move)] + qval)/(self.num_visits[bucb.tuplefy(move)]+1)
                self.num_visits[bucb.tuplefy(move)] += 1
                for move in self.UCBVals:
                    self.UCBVals[move] = self.qBar[bucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits)/self.num_visits[move])
                
            else:
                self.qBar[bucb.tuplefy(move)] = qval
                self.num_visits[bucb.tuplefy(move)] = 1
                self.UCBVals[bucb.tuplefy(move)] = self.qBar[bucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits))

            for i in self.qBar:
                if self.qBar[i] > self.qHat:
                    self.qHat = self.qBar[i]

            return(self.qHat)
        
        else:
            
            return(bucb.rollout(subposition))
            
    def rollout(position):
        
        board = checkers(position)
        
        for iter in range(30):
                
            board.updateAll()
            if board.win == 1:
                return(1)
    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
            board.updateAll()
            if board.win == 1:
                return(-1)    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
        
        board.updateLocation()
        return((len(board.pawns) + 1.5 * len(board.kings))/(len(board.pawns) + len(board.epawns) + 1.5 * len(board.kings) + 1.5 * len(board.ekings)))
    
    def starvingAction(self):
        
        maxUCB = 0
        maxAction = []
        
        for action in self.actions:
            
            if not (bucb.tuplefy(action) in self.children):
                
                return(action)

            if self.UCBVals[bucb.tuplefy(action)] > maxUCB:
                
                maxUCB = self.UCBVals[bucb.tuplefy(action)] 
                maxAction = action
        
        return(maxAction)
    
    def simulate(self, iter, p):
        
        self.actions = self.checkers.optimalActions
        
        if self.checkers.win == 1:
            
            return(self.checkers.optimalActions)
        
        for val in range(math.floor(iter/2)):

            self.update(self.starvingAction(), self.depth)
        
        actions = []
        
        for move in sorted(self.UCBVals)[len(self.actions) - math.floor(len(self.actions) * p):]:
            actions.append(bucb.untuplefy(move))
        
        self.actions = actions
        
        for val in range(iter - math.floor(iter/2)):
            
            self.update(self.starvingAction(), self.depth)        
        
        optscore = 0
        optmove = 0
        for i in self.qBar:
            if self.qBar[i] > optscore:
                optscore = self.qBar[i]
                optmove = i

        return(optmove)

    def traverse(self):
    
        temp = []
        temp.append(self.total_visits)
        
        if len(self.children) > 0:

            for child in self.children:
                
                temp.append(bucb.traverse(self.children[child]))

        return(temp)

IDUCB

In [4]:
import random
import math
from array import *
from copy import deepcopy
import numpy as np

graphs = []

class iducb:
    
    def __init__(self, position = [1] * 12 + [0] * 8 + [-1] * 12, explore_param = math.sqrt(2), maxdepth = 5, mindepth = 2, setdepth = False, depth = 0):
        
        self.checkers = checkers(position = position)
        self.checkers.updateAll()
        
        self.children = {}
        
        self.position = []
        self.explore_param = explore_param
        
        self.total_visits = 0
        self.num_visits = {}

        self.maxdepth = maxdepth
        self.mindepth = mindepth
        self.setdepth = setdepth
        self.depth = depth
        self.totaldepth = 0

        self.UCBVals = {}
        self.qBar = {}
        self.qHat = 0
        
        self.vHat = []
        self.optimalActions = []
                       
    def opponentAction(pawns, kings, epawns, ekings):
        
        flipState = checkers.flip(pawns, kings, epawns, ekings)
        flipState.updateAll()
        # print(flipState.optimalActions)
        return(random.choice(flipState.optimalActions))

    def tuplefy(move):
        
        newmove = []
        
        for step in move:
            
            newmove.append(tuple(step))
            
            if step == [""]:
                break
            
        return(tuple(newmove))
    
    def untuplefy(move):

        flag = True
        newmove = []
        
        for step in move:
            
            if list(step) == [""]:
                break
            
            newmove.append(list(step))
            
        newmove.append([""])
        
        return(newmove)
    
    def update(self, move, depth):
        
        """
        if isinstance(move[0], int):
            move = [move]    
           """
            
        if self.checkers.win == 1:
            return(1)
                
        position = deepcopy(self.checkers)
        position.updateBoard(move)

        opponentMove = iducb.opponentAction(position.pawns, position.kings, position.epawns, position.ekings)
        opponentBoard = checkers.flip(position.pawns, position.kings, position.epawns, position.ekings)
        opponentBoard.updateBoard(opponentMove)
        subposition = checkers.flip(opponentBoard.pawns, opponentBoard.kings, opponentBoard.epawns, opponentBoard.ekings)
        subposition.updateAll()

        if not self.setdepth:

            if not(iducb.tuplefy(move) in self.children):

                depth = self.maxdepth

            else:

                maxqBar = 0
                minqBar = 10

                for val in self.qBar:

                    if self.qBar[val] > maxqBar:
                        maxqBar = self.qBar[val]
                    if self.qBar[val] < minqBar:
                        minqBar = self.qBar[val]
                
                if self.qBar[iducb.tuplefy(move)] == maxqBar:

                    depth = self.maxdepth
                
                elif self.qBar[iducb.tuplefy(move)] == minqBar:

                    depth = self.mindepth
                
                else:

                    depth = round(np.random.beta((self.qBar[iducb.tuplefy(move)] - minqBar)/(maxqBar-self.qBar[iducb.tuplefy(move)]), 1) * (self.maxdepth - self.mindepth) + self.mindepth)
    
                self.totaldepth += depth

        if depth > 1:
            
            #print("blah")
                
            if iducb.tuplefy(move) in self.children:
                
                childState = self.children[iducb.tuplefy(move)]
                childState.position = subposition.position
                childState.checkers = checkers(position = childState.position)
                childState.checkers.updateAll()
            
            else:
            
                childState = iducb(position = subposition.position, explore_param = self.explore_param, setdepth = True, depth = depth-1)
                self.children[iducb.tuplefy(move)] = childState
            
            self.total_visits += 1
            
            childMove = childState.starvingAction()
            """
            if isinstance(childMove[0], int):
                childMove = [childMove]
            """
             
            qval = childState.update(childMove, depth - 1)
            
            if iducb.tuplefy(move) in self.UCBVals:
                self.qBar[iducb.tuplefy(move)] = (self.qBar[iducb.tuplefy(move)] * self.num_visits[iducb.tuplefy(move)] + qval)/(self.num_visits[iducb.tuplefy(move)]+1)
                self.num_visits[iducb.tuplefy(move)] += 1
                # self.UCBVals[ucb.tuplefy(move)] = self.qBar[ucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits))
                for move in self.UCBVals:
                    self.UCBVals[move] = self.qBar[iducb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits)/self.num_visits[move])
                    # print(move)
                
            else:
                self.qBar[iducb.tuplefy(move)] = qval
                self.num_visits[iducb.tuplefy(move)] = 1
                # self.UCBVals[ucb.tuplefy(move)] = self.qBar[ucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits))
                self.UCBVals[iducb.tuplefy(move)] = self.qBar[iducb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits)/self.num_visits[iducb.tuplefy(move)])
                
            for i in self.qBar:
                if self.qBar[i] > self.qHat:
                    self.qHat = self.qBar[i]
                    
            return(self.qHat)
        
        else:
            
            return(iducb.rollout(subposition))
            
    def rollout(position):
        
        board = checkers(position)
        
        for iter in range(40):
                
            board.updateAll()
            if board.win == 1:
                return(1)
    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
            board.updateAll()
            if board.win == 1:
                return(-1)    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
        
        board.updateLocation()
        # print(board.pawns, board.kings, board.epawns, board.ekings)
        return((len(board.pawns) + 1.5 * len(board.kings))/(len(board.pawns) + len(board.epawns) + 1.5 * len(board.kings) + 1.5 * len(board.ekings)))
    
    def starvingAction(self):
        
        maxUCB = 0
        maxAction = []
        
        for action in self.checkers.optimalActions:
                        
            if not (iducb.tuplefy(action) in self.children):
                
                return(action)

            if self.UCBVals[iducb.tuplefy(action)] > maxUCB:
                
                maxUCB = self.UCBVals[iducb.tuplefy(action)] 
                maxAction = action
        
        return(maxAction)
    
    def simulate(self):
        
        # print()
        # print(self.checkers.optimalActions)
        # print()
               
        if self.checkers.win == 1:
            
            return(self.checkers.optimalActions[0])
        
        for val in range(100):
            
            self.update(self.starvingAction(), self.depth)

        optscore = 0
        optmove = 0
        for i in self.qBar:
            if self.qBar[i] > optscore:
                optscore = self.qBar[i]
                optmove = i

        return(optmove)
            
    def traverse(self):

        temp = []
        temp.append(self.total_visits)
        
        if len(self.children) > 0:

            for child in self.children:
                
                temp.append(iducb.traverse(self.children[child]))
        print(temp)
        return(temp)

Avg Depth Per Move

In [34]:
import random
import datetime

depth = []

for x in range(100):

    f = random.randint(1, 12)
    e = random.randint(1, 12)

    fk = random.randint(0, f)
    fp = f - fk

    ek = random.randint(0, e)
    ep = e - ek

    possible = list(range(32))
    random.shuffle(possible)

    p = possible[:fp]
    e = possible[fp:fp+ep]
    kings = possible[fp+ep:fp+ep+fk]
    ekings = possible[fp+ep+fk:fp+ep+fk+ek]

    pawns = []
    epawns = []
    for val in p:
        if not (val > 27):
            pawns.append(val)
    for val in e:
        if not (val < 4):
            epawns.append(val)

    position = []
    for index in range(32):
        if index in pawns:
            position.append(1)
        elif index in kings:
            position.append(2)
        elif index in epawns:
            position.append(-1)
        elif index in ekings:
            position.append(-2)
        else:
            position.append(0)

    board = checkers(position = position)
    board.updateLocation()

    blahidu = iducb(position = board.position)

    blahidu.simulate()

    depth.append(blahidu.totaldepth)
depth

[429,
 480,
 431,
 375,
 475,
 475,
 433,
 457,
 487,
 395,
 433,
 495,
 484,
 458,
 0,
 475,
 365,
 495,
 377,
 447,
 443,
 401,
 490,
 437,
 436,
 495,
 475,
 478,
 471,
 0,
 439,
 490,
 495,
 446,
 199,
 443,
 468,
 495,
 487,
 472,
 487,
 495,
 479,
 485,
 461,
 495,
 460,
 495,
 421,
 437,
 420,
 495,
 472,
 439,
 434,
 471,
 408,
 460,
 441,
 440,
 473,
 454,
 495,
 0,
 458,
 410,
 484,
 455,
 373,
 495,
 424,
 495,
 476,
 408,
 495,
 466,
 469,
 475,
 495,
 490,
 405,
 490,
 495,
 451,
 469,
 490,
 490,
 199,
 421,
 481,
 427,
 453,
 472,
 466,
 430,
 399,
 389,
 467,
 433,
 459]

In [35]:
np.mean(depth)

437.97

Avg Time per Move

In [ ]:
import random
import datetime

u = []
bu = []
idu = []

for x in range(100):

    f = random.randint(1, 12)
    e = random.randint(1, 12)

    fk = random.randint(0, f)
    fp = f - fk

    ek = random.randint(0, e)
    ep = e - ek

    possible = list(range(32))
    random.shuffle(possible)

    p = possible[:fp]
    e = possible[fp:fp+ep]
    kings = possible[fp+ep:fp+ep+fk]
    ekings = possible[fp+ep+fk:fp+ep+fk+ek]

    pawns = []
    epawns = []
    for val in p:
        if not (val > 27):
            pawns.append(val)
    for val in e:
        if not (val < 4):
            epawns.append(val)

    position = []
    for index in range(32):
        if index in pawns:
            position.append(1)
        elif index in kings:
            position.append(2)
        elif index in epawns:
            position.append(-1)
        elif index in ekings:
            position.append(-2)
        else:
            position.append(0)

    board = checkers(position = position)
    board.updateLocation()

    blahu = ucb(position = board.position)
    blahbu = bucb(position = board.position)
    blahidu = iducb(position = board.position)

    prev = datetime.datetime.now()
    blahu.simulate()
    post = datetime.datetime.now()
    u.append((post - prev).total_seconds())

    prev = datetime.datetime.now()
    blahbu.simulate(100, 0.5)
    post = datetime.datetime.now()
    bu.append((post - prev).total_seconds())

    prev = datetime.datetime.now()
    blahidu.simulate()
    post = datetime.datetime.now()
    idu.append((post - prev).total_seconds())

print(np.mean(u))
print(np.mean(bu))
print(np.mean(idu))

In [36]:
print(np.mean(u))
print(np.mean(bu))
print(np.mean(idu))

2.14902007
2.1538892900000004
2.1163567000000003


In [30]:
idu

[2.117956,
 2.187574,
 2.306743,
 2.553013,
 0.230594,
 2.613253,
 2.209512,
 0.0,
 2.402144,
 2.384127,
 2.327058,
 2.330837,
 2.256724,
 2.449872,
 2.727351,
 2.70191,
 0.992048,
 0.428493,
 2.227027,
 2.090932,
 2.44426,
 2.145013,
 2.281632,
 1.845544,
 2.32498,
 2.609541,
 2.594944,
 2.20784,
 2.429916,
 2.468412,
 1.527527,
 2.666621,
 1.080348,
 2.351189,
 2.686356,
 0.0,
 0.820946,
 2.190325,
 2.306088,
 2.204469,
 2.229773,
 2.431967,
 2.230583,
 2.505074,
 2.586454,
 0.031117,
 2.521701,
 2.455995,
 0.812507,
 0.258977,
 2.630176,
 2.703907,
 2.487083,
 2.487119,
 2.413394,
 1.368849,
 2.552832,
 2.646474,
 2.637417,
 2.437138,
 2.610861,
 2.55387,
 2.772118,
 2.102845,
 2.540474,
 2.477383,
 1.046015,
 2.007722,
 2.057259,
 2.688624,
 2.553922,
 2.438989,
 2.263198,
 2.247863,
 2.332999,
 2.304565,
 2.527046,
 2.379432,
 2.222946,
 2.271966,
 2.581632,
 1.221801,
 2.506093,
 1.861645,
 1.995636,
 2.581564,
 2.274596,
 2.203415,
 2.275468,
 2.742611,
 2.212957,
 1.220396,
 0.

BUCB

In [13]:
import random
import math
from array import *
from copy import deepcopy
import numpy as np

graphs = []

class bucb:
    
    def __init__(self, position = [1] * 12 + [0] * 8 + [-1] * 12, explore_param = math.sqrt(2), depth = 5):
        
        self.checkers = checkers(position = position)
        self.checkers.updateAll()
        
        self.children = {}
        
        self.position = []
        self.explore_param = explore_param
        
        self.total_visits = 0
        self.num_visits = {}
        self.depth = depth
        
        self.UCBVals = {}
        self.qBar = {}
        self.qHat = 0
        
        self.vHat = []
        
        self.actions = []
        self.optimalActions = []
                        
    def opponentAction(pawns, kings, epawns, ekings):
        
        flipState = checkers.flip(pawns, kings, epawns, ekings)
        flipState.updateAll()
        return(random.choice(flipState.optimalActions))
    
    # todo: Move tuplefy out of ucb
    def tuplefy(move):
        
        newmove = []
        
        for step in move:
            
            newmove.append(tuple(step))
            
            if step == [""]:
                break
            
        return(tuple(newmove))
    
    def untuplefy(move):

        flag = True
        newmove = []
        
        for step in move:
            
            if list(step) == [""]:
                break
            
            newmove.append(list(step))
            
        newmove.append([""])
        
        return(newmove)
        
    def update(self, move, depth):

        if self.checkers.win == 1:
            return(1)
                
        position = deepcopy(self.checkers)
        position.updateBoard(move)

        opponentMove = bucb.opponentAction(position.pawns, position.kings, position.epawns, position.ekings)

        opponentBoard = checkers.flip(position.pawns, position.kings, position.epawns, position.ekings)
        opponentBoard.updateBoard(opponentMove)
        subposition = checkers.flip(opponentBoard.pawns, opponentBoard.kings, opponentBoard.epawns, opponentBoard.ekings)
        subposition.updateAll()

        if depth > 1:
                    
            if bucb.tuplefy(move) in self.children:
                
                childState = self.children[bucb.tuplefy(move)]
                childState.position = subposition.position
                childState.checkers = checkers(position = childState.position)
                childState.checkers.updateAll()
            
            else:
            
                childState = bucb(position = subposition.position, explore_param = self.explore_param, depth = self.depth-1)
                self.children[bucb.tuplefy(move)] = childState
            
            self.total_visits += 1
            
            childMove = childState.starvingAction()
            
            qval = childState.update(childMove, depth - 1)
            
            if bucb.tuplefy(move)in self.UCBVals:
                self.qBar[bucb.tuplefy(move)] = (self.qBar[bucb.tuplefy(move)] * self.num_visits[bucb.tuplefy(move)] + qval)/(self.num_visits[bucb.tuplefy(move)]+1)
                self.num_visits[bucb.tuplefy(move)] += 1
                for move in self.UCBVals:
                    self.UCBVals[move] = self.qBar[bucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits)/self.num_visits[move])
                
            else:
                self.qBar[bucb.tuplefy(move)] = qval
                self.num_visits[bucb.tuplefy(move)] = 1
                self.UCBVals[bucb.tuplefy(move)] = self.qBar[bucb.tuplefy(move)] + self.explore_param * math.sqrt(math.log(self.total_visits))

            for i in self.qBar:
                if self.qBar[i] > self.qHat:
                    self.qHat = self.qBar[i]

            return(self.qHat)
        
        else:
            
            return(bucb.rollout(subposition))
            
    def rollout(position):
        
        board = checkers(position)
        
        for iter in range(30):
                
            board.updateAll()
            if board.win == 1:
                return(1)
    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
            board.updateAll()
            if board.win == 1:
                return(-1)    
            board.updateBoard(random.choice(board.optimalActions))
            board = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
        
        board.updateLocation()
        return((len(board.pawns) + 1.5 * len(board.kings))/(len(board.pawns) + len(board.epawns) + 1.5 * len(board.kings) + 1.5 * len(board.ekings)))
    
    def starvingAction(self):
        
        maxUCB = 0
        maxAction = []
        
        for action in self.actions:
            
            if not (bucb.tuplefy(action) in self.children):
                
                return(action)

            if self.UCBVals[bucb.tuplefy(action)] > maxUCB:
                
                maxUCB = self.UCBVals[bucb.tuplefy(action)] 
                maxAction = action
        
        return(maxAction)
    
    def simulate(self, iter, p):
        
        self.actions = self.checkers.optimalActions
        
        if self.checkers.win == 1:
            
            return(self.checkers.optimalActions[0])
        
        for val in range(math.floor(iter/2)):

            self.update(self.starvingAction(), self.depth)
        
        actions = []

        if self.checkers.win == 1:
            
            return(self.checkers.optimalActions[0])

        for move in sorted(self.UCBVals)[len(self.actions) - math.floor(len(self.actions) * p):]:
            actions.append(bucb.untuplefy(move))
        
        self.actions = actions
        
        for val in range(iter - math.floor(iter/2)):
            
            self.update(self.starvingAction(), self.depth)        
        
        optscore = 0
        optmove = 0
        for i in self.qBar:
            if self.qBar[i] > optscore:
                optscore = self.qBar[i]
                optmove = i

        return(optmove)

    def traverse(self):
    
        temp = []
        temp.append(self.total_visits)
        
        if len(self.children) > 0:

            for child in self.children:
                
                temp.append(bucb.traverse(self.children[child]))

        return(temp)

Simulator

In [19]:
import time
for val in range(10):
    board = checkers(position = [0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,0,2,2,0,0])
    # board = checkers(position = [0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, -1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    # board = checkers(position = [1] * 8 + [0] * 16 + [-1]*8)
    # board = checkers(position = [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2])
    # board = checkers()
    board.updateAll()
    blah = bucb(position = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings).position)
    #blah = bucb(position = board.position)

    bleh = checkers.flip(board.pawns, board.kings, board.epawns, board.ekings)
    bleh.updateLocation()
    # board.updateLocation()
    bleh.displayBoard()
    # board.displayBoard()
    
    x = blah.simulate(25, .5)
    print(x)
    x = bucb.untuplefy(x)
    print(x)
    # bleh.displayBoard()
    # print(board.pawns, board.kings, board.epawns, board.ekings)
    #board.updateBoard(x)
    # bleh.updateBoard(x)

-	-	-	-	-	-2	-	-2

-	-	-1	-	-1	-	-	- 

-	-	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

-	-	-	-	-	-1	-	-

-1	-	-	-	-1	-	-	- 

-	-1	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

()
[['']]
-	-	-	-	-	-2	-	-2

-	-	-1	-	-1	-	-	- 

-	-	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

-	-	-	-	-	-1	-	-

-1	-	-	-	-1	-	-	- 

-	-1	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

()
[['']]
-	-	-	-	-	-2	-	-2

-	-	-1	-	-1	-	-	- 

-	-	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

-	-	-	-	-	-1	-	-

-1	-	-	-	-1	-	-	- 

-	-1	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

()
[['']]
-	-	-	-	-	-2	-	-2

-	-	-1	-	-1	-	-	- 

-	-	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

-	-	-	-	-	-1	-	-

-1	-	-	-	-1	-	-	- 

-	-1	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

()
[['']]
-	-	-	-	-	-2	-	-2

-	-	-1	-	-1	-	-	- 

-	-	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

-	-	-	-	-	-1	-	-

-1	-	-	-	-1	-	-	- 

-	-1	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

(('',),)
[['']]
-	-	-	-	-	-2	-	-2

-	-	-1	-	-1	-	-	- 

-	-	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

-	-	-	-	-	-1	-	-

-1	-	-	-	-1	-	-	- 

-	-1	-	-1	-	-	-	-

-1	-	-	-	-	-	-	- 

(('',),)
[['']]
-	-	-	-	-	-2	-	-

: 

In [240]:
blah.children

{((1, 17, 5, 9), ('',)): <__main__.ucb at 0x26c106ddbe0>}

In [121]:
blah.total_visits

1000

In [123]:
for child in blah.children:
    print(blah.children[child].total_visits)

149
149
132
142
130
149
149


In [70]:
print(blah.traverse())

[0]
[1, [0]]
[1, [1, [0]]]
[1, [1, [1, [0]]]]
[0]
[1, [0]]
[1, [1, [0]]]
[1, [1, [1, [0]]]]
[0]
[1, [0]]
[1, [1, [0]]]
[1, [1, [1, [0]]]]
[0]
[1, [0]]
[1, [1, [0]]]
[0]
[1, [0]]
[1, [1, [0]]]
[0]
[1, [0]]
[1, [1, [0]]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[0]
[0]
[0]
[135, [0], [0], [0], [0]]
[0]
[1, [0]]
[141, [1, [0]], [1, [0]], [1, [0]], [1, [0]], [1, [0]], [135, [0], [0], [0], [0]], [1, [0]]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[15, [0]]
[0]
[1, [0]]
[0]
[1, [0]]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[23, [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
[0]
[1, [0]]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[27, [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
[0]
[1, [0]]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[79, [0], [0], [0], [0], [0], [0], [0], [0], [0]]
[0]
[1, [0]]
[153, [1, [0]], [1, [0]], [1, [0]], [1, [0]], [15, [0]], [1, [0]], [1, [0]], [23, [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], [1, [0]], 

In [71]:
nextlvl = []
for val in [94, [1, [1, [0]]], [1, [1, [0]]], [1, [1, [0]]], [73, [2, [0], [0]], [1, [0]], [7, [0], [0], [0], [0], [0], [0], [0]], [1, [0]], [1, [0]], [4, [0], [0], [0], [0]], [27, [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], [1, [0]], [1, [0]], [1, [0]], [1, [0]], [1, [0]], [1, [0]], [8, [0], [0], [0], [0], [0], [0], [0], [0]], [9, [0], [0], [0], [0], [0], [0], [0]], [7, [0], [0], [0], [0], [0], [0], [0]]], [1, [1, [0]]], [1, [1, [0]]], [14, [1, [0]], [1, [0]], [1, [0]], [1, [0]], [1, [0]], [1, [0]], [8, [0], [0], [0], [0], [0], [0]]], [1, [1, [0]]], [1, [1, [0]]]]:
    if not isinstance(val, list):
        print(val)
    else:
        nextlvl.append(val)

print()

nextnextlvl = []
for val in nextlvl:
    flag = True
    for item in val:
        if not isinstance(item, list):
            print(item)
            flag = False
            # print(item)


94

1
1
1
73
1
1
14
1
1
